Este notebook está sendo utilizado para investigar o estado dos dados e processá-los, a fim de realizar posteriormente uma análise exploratória dos dados e em seguida treinar um modelo de machine learning para prever se uma ação está barata, cara ou se encontra em um ponto neutro baseando nos dados dos indicadores financeiros da empresa.

Este notebook servirá como base para criar um procedimento que será utilizado para todos os tickers, consolidando os dados em um único arquivo. Esse processo estará no arquivo .py (src/data_processing/processor.py) que automatizará essa tarefa.

In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import yfinance as yf
import os

In [1]:
#Define um ticker para processar os dados
ticker = "VALE3"

#Intervalo de data para buscar os preços da ação no Yahoo Finance
start_date = '2007-01-01'
end_date = '2024-01-01'

# Carregar dados
data = pd.read_csv(f"../../data/raw/indicadores_{ticker}.csv")

# Baixar os dados do Yahoo Finance
precos = yf.download((ticker+'.SA'), start=start_date, end=end_date, interval="1wk")

[*********************100%%**********************]  1 of 1 completed


In [2]:
data.head()

,Tipo do Indicador,Nome do Indicador,ATUAL,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
0,INDICADORES DE VALUATION,D.Y,"11,75%","7,87%","8,53%","18,79%","2,75%","2,65%","3,86%","3,29%","0,65%","7,49%","8,63%","5,06%","5,35%","7,31%",-%,-%,-%
1,INDICADORES DE VALUATION,P/L,"5,68","7,37","4,43","3,30","17,30","-42,22","10,50","11,98","10,12","-1,55","123,17","1.664,73","22,93","5,60",-,-,-
2,INDICADORES DE VALUATION,PEG RATIO,"-0,10","-0,13",-,-,-,-,"0,23","0,37","-0,08","0,00","0,17","-16,84","-0,31",-,-,-,-
3,INDICADORES DE VALUATION,P/VP,"1,42","1,84","2,27","2,08","2,49","1,74","1,58","1,47","1,06","0,52","0,80","1,29","1,51","1,48",-,-,-
4,INDICADORES DE VALUATION,EV/EBITDA,"4,04","5,05","4,06","2,70","6,34","15,85",-,-,-,-,-,-,-,-,-,-,-


- O arquivo atualmente apresenta uma estrutura onde cada ano é representado por uma coluna, porém, precisamos reorganizar para que os anos sejam representados em linhas.

- Além disso, os valores numéricos estão formatados com caracteres especiais, como por exemplo "11,75%", e será necessário ajustá-los para o formato padrão, como "11.75".

- Por fim, em vez de utilizar o caractere "-" para representar valores nulos, precisaremos substituí-lo por "null", e remover a coluna "Tipo do Indicador". Também será necessário renomear a coluna atual para "2024".

In [3]:
precos_df = pd.DataFrame(precos)

precos_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-01-01,31.850000,32.700001,29.100000,29.299999,12.824647,11537400
2007-01-08,29.500000,31.870001,29.004999,31.799999,13.918905,10284800
2007-01-15,31.900000,32.369999,30.850000,32.360001,14.164014,9608800
2007-01-22,32.450001,35.799999,32.375000,35.200001,15.407085,9828600
2007-01-29,35.375000,36.404999,34.275002,35.599998,15.582168,9328800


In [4]:
# Função para calcular a média do preço de fechamento para um determinado ano
def close_price_mean_for_year(df, year):
    df_year = df[df.index.year == year]
    return df_year['Close'].mean()

In [5]:
# Remover a coluna "Tipo do Indicador"
data = data.drop('Tipo do Indicador', axis=1)

# Renomear a coluna "ATUAL" para "2024"
data = data.rename(columns={'ATUAL': '2024'})

In [6]:
df = pd.DataFrame(data)

# Remover caracteres especiais e ajustar o separador decimal
for col in df.columns[1:]:
    df[col] = df[col].str.replace(",", ".").str.rstrip("%")


# Substituir '-' por null em todas as colunas exceto as duas primeiras
df.iloc[:, 1:] = df.iloc[:, 1:].replace('-', np.nan)


In [7]:
# Converter as colunas para tipo numérico
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.head()

,Nome do Indicador,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
0,D.Y,11.75,7.87,8.53,18.79,2.75,2.65,3.86,3.29,0.65,7.49,8.63,5.06,5.35,7.31,NaN,NaN,NaN
1,P/L,5.68,7.37,4.43,3.30,17.30,-42.22,10.50,11.98,10.12,-1.55,123.17,NaN,22.93,5.60,NaN,NaN,NaN
2,PEG RATIO,-0.10,-0.13,NaN,NaN,NaN,NaN,0.23,0.37,-0.08,0.00,0.17,-16.84,-0.31,NaN,NaN,NaN,NaN
3,P/VP,1.42,1.84,2.27,2.08,2.49,1.74,1.58,1.47,1.06,0.52,0.80,1.29,1.51,1.48,NaN,NaN,NaN
4,EV/EBITDA,4.04,5.05,4.06,2.70,6.34,15.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Transpor o DataFrame
df_transposed = df.transpose()

# Resetar o índice
df_transposed = df_transposed.reset_index()

# Definir a primeira linha como o cabeçalho
df_transposed.columns = df_transposed.iloc[0]

# Remover a primeira linha do DataFrame
df_transposed = df_transposed[1:]

# Resetar o índice
df_transposed = df_transposed.reset_index(drop=True)

#Troca o nome da coluna Nome do Indicador para Ano
df_transposed = df_transposed.rename(columns={'Nome do Indicador': 'Ano'})

df_transposed

,Ano,D.Y,P/L,PEG RATIO,P/VP,EV/EBITDA,EV/EBIT,P/EBITDA,P/EBIT,VPA,...,M. BRUTA,M. EBITDA,M. EBIT,M. LÍQUIDA,ROE,ROA,ROIC,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS
0,2024,11.75,5.68,-0.1,1.42,4.04,5.0,3.4,4.21,41.9,...,40.12,38.45,31.07,23.03,24.99,10.66,21.26,0.46,15.85,21.94
1,2023,7.87,7.37,-0.13,1.84,5.05,6.25,4.42,5.47,41.9,...,40.12,38.45,31.07,23.03,24.99,10.66,21.26,0.46,15.85,21.94
2,2022,8.53,4.43,NaN,2.27,4.06,5.16,3.7,4.7,39.15,...,45.17,50.73,39.88,42.35,51.27,21.16,28.76,0.5,15.85,40.33
3,2021,18.79,3.3,NaN,2.08,2.7,2.91,2.63,2.83,37.49,...,60.05,51.84,48.15,41.3,63.01,24.29,42.35,0.59,25.41,55.55
4,2020,2.75,17.3,NaN,2.49,6.34,9.03,6.23,8.88,35.16,...,52.73,35.57,24.97,12.81,14.38,5.59,18.93,0.44,21.72,NaN
5,2019,2.65,-42.22,NaN,1.74,15.85,139.25,14.32,125.75,30.56,...,43.6,13.24,1.51,-4.49,-4.13,-1.8,-0.12,0.4,12.46,NaN
6,2018,3.86,10.5,0.23,1.58,NaN,7.11,NaN,6.24,32.25,...,39.62,NaN,32.1,19.08,15.06,7.51,18.07,0.39,5.79,194.87
7,2017,3.29,11.98,0.37,1.47,NaN,7.85,NaN,6.11,27.41,...,38.03,NaN,31.82,16.24,12.26,5.37,13.45,0.33,3.53,12.25
8,2016,0.65,10.12,-0.08,1.06,NaN,10.44,NaN,6.5,24.26,...,35.39,NaN,21.89,14.07,10.46,4.13,4.86,0.29,-1.21,-18.85
9,2015,7.49,-1.55,0.0,0.52,NaN,-5.83,NaN,-2.39,25.01,...,19.57,NaN,-36.67,-56.64,-33.71,-12.79,-19.03,0.23,-1.27,NaN


In [9]:
#adicionando o ticker
df_transposed['Ticker'] = ticker

#seta o indice como o ticker e ano
#df_transposed = df_transposed.set_index(['Ticker','Ano'])

df_transposed

,Ano,D.Y,P/L,PEG RATIO,P/VP,EV/EBITDA,EV/EBIT,P/EBITDA,P/EBIT,VPA,...,M. EBITDA,M. EBIT,M. LÍQUIDA,ROE,ROA,ROIC,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,Ticker
0,2024,11.75,5.68,-0.1,1.42,4.04,5.0,3.4,4.21,41.9,...,38.45,31.07,23.03,24.99,10.66,21.26,0.46,15.85,21.94,VALE3
1,2023,7.87,7.37,-0.13,1.84,5.05,6.25,4.42,5.47,41.9,...,38.45,31.07,23.03,24.99,10.66,21.26,0.46,15.85,21.94,VALE3
2,2022,8.53,4.43,NaN,2.27,4.06,5.16,3.7,4.7,39.15,...,50.73,39.88,42.35,51.27,21.16,28.76,0.5,15.85,40.33,VALE3
3,2021,18.79,3.3,NaN,2.08,2.7,2.91,2.63,2.83,37.49,...,51.84,48.15,41.3,63.01,24.29,42.35,0.59,25.41,55.55,VALE3
4,2020,2.75,17.3,NaN,2.49,6.34,9.03,6.23,8.88,35.16,...,35.57,24.97,12.81,14.38,5.59,18.93,0.44,21.72,NaN,VALE3
5,2019,2.65,-42.22,NaN,1.74,15.85,139.25,14.32,125.75,30.56,...,13.24,1.51,-4.49,-4.13,-1.8,-0.12,0.4,12.46,NaN,VALE3
6,2018,3.86,10.5,0.23,1.58,NaN,7.11,NaN,6.24,32.25,...,NaN,32.1,19.08,15.06,7.51,18.07,0.39,5.79,194.87,VALE3
7,2017,3.29,11.98,0.37,1.47,NaN,7.85,NaN,6.11,27.41,...,NaN,31.82,16.24,12.26,5.37,13.45,0.33,3.53,12.25,VALE3
8,2016,0.65,10.12,-0.08,1.06,NaN,10.44,NaN,6.5,24.26,...,NaN,21.89,14.07,10.46,4.13,4.86,0.29,-1.21,-18.85,VALE3
9,2015,7.49,-1.55,0.0,0.52,NaN,-5.83,NaN,-2.39,25.01,...,NaN,-36.67,-56.64,-33.71,-12.79,-19.03,0.23,-1.27,NaN,VALE3


In [10]:
df_transposed['D.Y'] = df_transposed['D.Y'].astype(float).fillna(0)


In [11]:
# Aplicar a função para cada linha (ano) do DataFrame
df_transposed['PrecoAnoSeguinte'] = df_transposed['Ano'].astype(int).apply(lambda x: close_price_mean_for_year(precos_df, x + 1))
df_transposed['PrecoAnoAtual'] = df_transposed['Ano'].astype(int).apply(lambda x: close_price_mean_for_year(precos_df, x))

In [12]:
df_transposed

,Ano,D.Y,P/L,PEG RATIO,P/VP,EV/EBITDA,EV/EBIT,P/EBITDA,P/EBIT,VPA,...,M. LÍQUIDA,ROE,ROA,ROIC,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,Ticker,PrecoAnoSeguinte,PrecoAnoAtual
0,2024,11.75,5.68,-0.1,1.42,4.04,5.0,3.4,4.21,41.9,...,23.03,24.99,10.66,21.26,0.46,15.85,21.94,VALE3,NaN,NaN
1,2023,7.87,7.37,-0.13,1.84,5.05,6.25,4.42,5.47,41.9,...,23.03,24.99,10.66,21.26,0.46,15.85,21.94,VALE3,NaN,73.996923
2,2022,8.53,4.43,NaN,2.27,4.06,5.16,3.7,4.7,39.15,...,42.35,51.27,21.16,28.76,0.5,15.85,40.33,VALE3,73.996923,80.667885
3,2021,18.79,3.3,NaN,2.08,2.7,2.91,2.63,2.83,37.49,...,41.3,63.01,24.29,42.35,0.59,25.41,55.55,VALE3,80.667885,94.930192
4,2020,2.75,17.3,NaN,2.49,6.34,9.03,6.23,8.88,35.16,...,12.81,14.38,5.59,18.93,0.44,21.72,NaN,VALE3,94.930192,57.694231
5,2019,2.65,-42.22,NaN,1.74,15.85,139.25,14.32,125.75,30.56,...,-4.49,-4.13,-1.8,-0.12,0.4,12.46,NaN,VALE3,57.694231,49.491346
6,2018,3.86,10.5,0.23,1.58,NaN,7.11,NaN,6.24,32.25,...,19.08,15.06,7.51,18.07,0.39,5.79,194.87,VALE3,49.491346,49.973774
7,2017,3.29,11.98,0.37,1.47,NaN,7.85,NaN,6.11,27.41,...,16.24,12.26,5.37,13.45,0.33,3.53,12.25,VALE3,49.973774,31.257884
8,2016,0.65,10.12,-0.08,1.06,NaN,10.44,NaN,6.5,24.26,...,14.07,10.46,4.13,4.86,0.29,-1.21,-18.85,VALE3,31.257884,17.637308
9,2015,7.49,-1.55,0.0,0.52,NaN,-5.83,NaN,-2.39,25.01,...,-56.64,-33.71,-12.79,-19.03,0.23,-1.27,NaN,VALE3,17.637308,18.495000


In [13]:
# Criar o campo alvo
df_transposed['Alvo'] = np.where(df_transposed['PrecoAnoSeguinte'] > df_transposed['PrecoAnoAtual'] * 1.15, 'Barata',
                        np.where(df_transposed['PrecoAnoSeguinte'] < df_transposed['PrecoAnoAtual'] * 0.85, 'Cara', 'Neutra'))

In [14]:
df_transposed

,Ano,D.Y,P/L,PEG RATIO,P/VP,EV/EBITDA,EV/EBIT,P/EBITDA,P/EBIT,VPA,...,ROE,ROA,ROIC,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,Ticker,PrecoAnoSeguinte,PrecoAnoAtual,Alvo
0,2024,11.75,5.68,-0.1,1.42,4.04,5.0,3.4,4.21,41.9,...,24.99,10.66,21.26,0.46,15.85,21.94,VALE3,NaN,NaN,Neutra
1,2023,7.87,7.37,-0.13,1.84,5.05,6.25,4.42,5.47,41.9,...,24.99,10.66,21.26,0.46,15.85,21.94,VALE3,NaN,73.996923,Neutra
2,2022,8.53,4.43,NaN,2.27,4.06,5.16,3.7,4.7,39.15,...,51.27,21.16,28.76,0.5,15.85,40.33,VALE3,73.996923,80.667885,Neutra
3,2021,18.79,3.3,NaN,2.08,2.7,2.91,2.63,2.83,37.49,...,63.01,24.29,42.35,0.59,25.41,55.55,VALE3,80.667885,94.930192,Cara
4,2020,2.75,17.3,NaN,2.49,6.34,9.03,6.23,8.88,35.16,...,14.38,5.59,18.93,0.44,21.72,NaN,VALE3,94.930192,57.694231,Barata
5,2019,2.65,-42.22,NaN,1.74,15.85,139.25,14.32,125.75,30.56,...,-4.13,-1.8,-0.12,0.4,12.46,NaN,VALE3,57.694231,49.491346,Barata
6,2018,3.86,10.5,0.23,1.58,NaN,7.11,NaN,6.24,32.25,...,15.06,7.51,18.07,0.39,5.79,194.87,VALE3,49.491346,49.973774,Neutra
7,2017,3.29,11.98,0.37,1.47,NaN,7.85,NaN,6.11,27.41,...,12.26,5.37,13.45,0.33,3.53,12.25,VALE3,49.973774,31.257884,Barata
8,2016,0.65,10.12,-0.08,1.06,NaN,10.44,NaN,6.5,24.26,...,10.46,4.13,4.86,0.29,-1.21,-18.85,VALE3,31.257884,17.637308,Barata
9,2015,7.49,-1.55,0.0,0.52,NaN,-5.83,NaN,-2.39,25.01,...,-33.71,-12.79,-19.03,0.23,-1.27,NaN,VALE3,17.637308,18.495000,Neutra


In [15]:
# Verificar se o diretório existe, caso contrário, criar
output_dir = '../../data/processed'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Salvar o DataFrame em um arquivo parquet
output_file = os.path.join(output_dir, f'dados_{ticker}.parquet')
df_transposed.to_parquet(output_file)